# Исследование объявлений о продаже квартир

В нашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. 

Наша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

### Откроем файл с данными и изучим общую информацию. 

In [1]:
import pandas as pd

In [2]:
# После прочтения файла было обнаружено, что разделялись табуляцей, что и было добавлен параметр sep
# data = pd.read_csv('/datasets/real_estate_data.csv', sep = '\t')
data = pd.read_csv('datasets/real_estate_data.csv', sep = '\t')
data.info()

#### Получаем информацию и ознакомимся

In [3]:
data.head(100)

Названия колонки в порядке, нет никаких посимвольных ошибок

Но не будем пренебергать числами, которые не совпадают с размером датафрейма, например, данные "is_apartment" начисляется 2775 как заполненные, а остальные (23699 - 2775 = 20924) не заполнены. Можно подтвердиться решением кода внизу:

In [4]:
data['is_apartment'].isna().sum()

Как и остальные данные также будут претерпевать изменениями - предобработками данных

#### Построение гистограмм

In [5]:
data.hist(figsize=(15, 20))

### Предобработка данных

#### Удаление пропусков

In [6]:
data.isna().sum()

Можно заметить, что в столбцах есть пропущения: 
- ceiling_height - "высота потолка", 

- floors_total - "всего этажей в доме"

- living_area - "жилая площадь в квадратных метрах (м²)", 

- is_apartment - "апартаменты (булев тип)", 

- kitchen_area - "площадь кухни в квадратных метрах (м²)", 

- balcony - "количество балкона", 

- locality_name - "название населённого пункта", 

- airports_nearest - "расстояние до ближайшего аэропорта в метрах (м)", 

- cityCenters_nearest - "расстояние до центра города (м)", 

- parks_around3000 - "число парков в радиусах 3 км"

- parks_nearest - "расстояние до ближайшего парка (м)"

- ponds_around3000 - "число водоёмов в радиусе 3 км"

- ponds_nearest - "расстояние до ближайшего водоёма (м)"

- days_exposition - "сколько дней было размещено объявление (от публикации до снятия)"

Идея и действия с пропусками:

- Высоту потолка можно поставить медианное значение

- ~~Кол-во этажей в доме можно поставить медианное значение~~
- Кол-во этажей оставим пустыми, но сделать его целочисленными типами

- Жилую площадку также можно поставить медианное значение

- ~~Аппартаменты можно по умолчанию ставить 0, т.е. данная квартира не является аппартаментом~~

- С аппартаментом спорный вопрос, лучше оставить их пустыми (т.к. эти данные являются категоризациями). Сделать его булевыми типами

- Площадь кухни можно поставить медианное значение

- По рекомендацию количество балкона можно поставить как 0. Но сделать его целочисленными типами

- Пустое значение населенного пункта оставить как и есть (т.к. является катогеризацией), а неявные дубликаты устраняем

- Расстояние до ближайшего аэропорта оставим пустым, т.к. неизвестно где располагается недвижимость

- Расстояние до центра города аналогично с расстоянием до ближайшего аэропорта

- Число парков можно аналогично с предыдущим пунктом. Но сделать его целочисленными типами

- Расстояние ближайшего парка аналогично с предыдущим пунктом

- Число водоемов можно аналогично с предыдущим пунктом. Но сделать его целочисленными типами

- Расстояние до ближайшего водоема оставим пустым, т.к. неизвестно где располагается недвижимость

- Срок публикации аналогично оставим пустым, т.к. у каждого дома есть свой уникальный срок публикации, поэтому нельзя полагаться на медианное значение и любые приближенные значения

В проектах ГК ПИК высота потолков в квартирах находится в диапазоне 2,6 - 3,1 м, замечает Сергей Ковров.

Однако максимальная высота потолков, как правило, не регламентируется,
но чаще всего она в современной застройке не превышает 3,2 метра. 
В старых домах с увеличенными габаритами и площадью потолки могут быть 4 метра высотой и более.

In [82]:
# Смотрим, какая высота потолка самая распространенная
data['ceiling_height'].value_counts()

Здесь получилось, что распространенной высоты потолка является 2,5 

In [8]:
# Не все данные показывают, узнаем через метод unique
print(data['ceiling_height'].sort_values().unique())

Но можно заметить, что существуют дома, потолки которого превышают не только 3,2 метра, но и меньше 2,5 метров. Устраним их

In [9]:
# Устраняем данные дома, в котором высота потолки не находится в этом диапазоне 2,5 - 3,2, 
# но за исключением выше 24, 25, 26, 27, 27,5, 32,0.
data = data[((data['ceiling_height'] > 2.5) & (data['ceiling_height'] <= 3.2)) | (data['ceiling_height'] >= 24)]

# Уменьшаем в 10 раз для домов, у которых потолки выше 24 метров
data.loc[data['ceiling_height'] >= 24, 'ceiling_height'] = data['ceiling_height'] / 10

# Устраняем дом, у которого потолок 10 метр
data = data[data['ceiling_height'] != 10]

In [10]:
# Проверим и убедимся, что аномальных значений нет
print(data['ceiling_height'].sort_values().unique())

In [11]:
# Заполним пропуски медианные значения высоты потолка
print('Медианное значение высоты потолка:', data['ceiling_height'].median())
data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].median())

In [12]:
# Заполним пропуски медианные значения жилой площади
print('Медианное значение:', data['living_area'].median())
data['living_area'] = data['living_area'].fillna(data['living_area'].median())

data['living_area'].value_counts()

In [13]:
# Заполним пропуски медианные значения площади кухни
print('Медианное значение площади кухни:', data['kitchen_area'].median())
data['kitchen_area'] = data['kitchen_area'].fillna(data['kitchen_area'].median())

data['kitchen_area'].value_counts()

In [14]:
# Обнаружили, что кол-во балконов является вещественными числами, их нужно сделать целочисленными типами
# Заполним пропуски количеств балконов как отстутствие (0)
data['balcony'] = data['balcony'].fillna(0).astype('Int64')

data['balcony'].value_counts()

In [15]:
# Посчитаем дубликаты
data['locality_name'] = data['locality_name'].str.lower()
data['locality_name'].duplicated().sum()

#### Изменение данных названий населенного пункта

In [16]:
# Получим уникальные названий населенного пункта
data['locality_name'].sort_values().unique()

Создадим новую колонку, в которой находятся только названия без дополнительных пояснений вроде того "поселок" и т.д.

In [17]:
def locality_name(row):
    try:
        if 'городской поселок' in row:
            return row.replace('городской поселок ', '')
        elif 'городской посёлок' in row:
            return row.replace('городской посёлок ', '')
        elif 'деревня' in row:
            return row.replace('деревня ', '')
        elif 'поселок городского типа' in row:
            return row.replace('поселок городского типа ', '')
        elif 'посёлок городского типа' in row:
            return row.replace('посёлок городского типа ', '')
        elif 'поселок' in row:
            return row.replace('поселок ', '')
        elif 'посёлок' in row:
            return row.replace('посёлок ', '')
        elif 'село' in row:
            return row.replace('село ', '')
        elif 'садовое товарищество ' in row:
            return row.replace('садовое товарищество ', '')
        elif 'nan' in row:
            return 'Неизвестное'
        else:
            return row
    except:
        return 'нет категории'

data['locality_good_name'] = data['locality_name'].apply(locality_name)

In [18]:
# Получим уникальные названий населенного пункта
data['locality_good_name'].sort_values().unique()

In [19]:
# Проверим и убедимся, что заполнили пропущенные значения
data['locality_good_name'].isna().sum()

Можно заметить, что колонка is_apartment не является булевым типом, следовательно, нужно исправить её

In [20]:
data['is_apartment'] = data['is_apartment'].astype('bool')

In [21]:
# Проверим, есть ли в других местах False
data['is_apartment'].value_counts()

In [22]:
# Кол-во этажей является вещественными числами, их нужно сделать целочисленными типами
data['floors_total'] = data['floors_total'].astype('Int64')

In [23]:
# Число парков в радиусах 3 км является вещественными числами, их нужно сделать целочисленными типами
data['parks_around3000'] = data['parks_around3000'].astype('Int64')

In [24]:
# Число водоемов в радиусах 3 км является вещественными числами, их нужно сделать целочисленными типами
data['ponds_around3000'] = data['ponds_around3000'].astype('Int64')

In [25]:
# Сэкономим память, изменив тип вещественного числа с float64 на float32
data['total_area'] = data['total_area'].astype('Float32')

In [26]:
# Проверим и убедимся, что заполнили пропущенные значения в некоторых колонках
data.isna().sum()

### Посчитайте и добавьте в таблицу новые столбцы

In [27]:
data.head()

In [28]:
# Цена одного квадратного метра = цена на момент снятия с публикации / общая площадь квартиры в квадратных метрах (м²)

data['price_of_one_square_meter'] = data['last_price'] / data['total_area']
data['price_of_one_square_meter']

In [29]:
# Преобразуем в правильный формат даты
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format = '%Y/%m/%d %H:%M:')

In [30]:
# Конвентируем в дни недели - дни публикации
data['day_of_publication'] = data['first_day_exposition'].dt.day_name()

# Конвентируем в дни недели - месяц публикации
data['month_of_publication'] = data['first_day_exposition'].dt.month_name()

# Конвентируем в дни недели - год публикации
data['year_of_publication'] = data['first_day_exposition'].dt.year

In [31]:
data['type_floor'] = 'Другой'

# Первый
data.loc[data['floor'] == 1, 'type_floor'] = 'Первый'

# Последний
data.loc[data['floor'] == data['floors_total'], 'type_floor'] = 'Последний'

In [32]:
# Проверяем и убедимся, что в данных содержатся 3 типа
data['type_floor'].unique()

In [33]:
# Расстояние до центра города в километрах
data['cityCenters_nearest_km'] = round(data['cityCenters_nearest'] / 1000)

In [34]:
# Здесь можно заметить, что существуют данные, в которых количество комната равно нулю
data[data['rooms'] == 0].head()

In [35]:
# Это является ошибочной, заменим 0 на NaN
data.loc[data['rooms'] == 0, 'rooms'] = None
data['rooms'] = data['rooms'].astype('Int64')

In [36]:
# Проверим на пропущенные значения
data['rooms'].isna().sum()

In [37]:
# Проверяем
data.head()

In [38]:
# Все данные подправлены и заполнены, теперь можно приступать к исследованию
data.info()

### Проведите исследовательский анализ данных

In [39]:
data['total_area'].plot(kind = 'hist', figsize = (15, 10), grid = True)

Можно заметить, что площадь ~100 м^2 является огромной количествой в домах

In [40]:
data['living_area'].plot(kind = 'hist', figsize = (15, 10), grid = True)

Можно заметить, что здесь жилая площадь 0-40 м^2 является огромной количествой в домах, но 50-70 м^2 также обладает немалым количеством

In [41]:
data['kitchen_area'].plot(kind = 'hist', figsize = (15, 10), grid = True)

Можно заметить, что здесь кухонная площадь 0-10 м^2 является огромной количествой в домах, но 10-25 м^2 также обладает немалым количеством. Нельзя отрицать, что существуют дома, которые есть кухня с площадью более 25 м^2

In [42]:
data['last_price'].value_counts().sort_values(ascending = False)

In [43]:
data['rooms'].value_counts().sort_values(ascending = False)

In [44]:
data['rooms'].plot(kind = 'hist', figsize = (15, 10), grid = True)

Можно заметить, что здесь часто всего встречается двухкомнатный дом, но 2 место по количеству комнат занимает однокомнатный дом

In [45]:
data['ceiling_height'].plot(kind = 'hist', figsize = (15, 10), grid = True)

Можно заметить, что здесь самая распростренненая высота потолка является ~2,7 метра. Самая редкая высота - ~2,9 метра

In [46]:
data['floor'].plot(kind = 'hist', figsize = (15, 10), grid = True)

Можно заметить, что здесь самое распространенное предложение приобрести комнату, у которой этаж не более 4

In [47]:
data['type_floor'].hist(figsize = (15, 10))

Можно заметить, что здесь самый предложенный вариант дома, этаж у которого не является ни первого, ни последнего. Однако последний этаж дома по количеству предложенного варианта не уступает первому этажу

In [48]:
data['floors_total'].plot(kind = 'hist', figsize = (15, 10), grid = True)

Можно заметить, что здесь самый распростренненый тип дома, у которого 10 этажей. Самый редкий - более 30 этажей

In [49]:
data['cityCenters_nearest_km'].plot(kind = 'hist', figsize = (15, 10), grid = True)

Можно заметить, что здесь существуют много дома, который располагается около 10 км от центра города. А самый редкий - около 45 и более 55 км от центра города

In [50]:
data['airports_nearest'].plot(kind = 'hist', figsize = (15, 10), grid = True)

Можно заметить, что здесь часто встречается дом, расстояние которого до аэропорта около 20 км. Но нелья отрицать, что есть внушительные количество домов, расстояние которых до аэропорта около 15 км и 40 км. Самый редкое - более 60 км.

In [51]:
data['parks_nearest'].plot(kind = 'hist', figsize = (15, 10), grid = True)

Можно заметить, что здесь часто встречается дом, расстояние которого до ближайшего парка около 0,5 км. Второе место по количеству занимает дома, у которого расстояние в шаговой доступности - около 0,25 км. 
Однако существуют малое количество дома, расстояние которого до ближайшего парка более 1 км.

In [52]:
data['day_of_publication'].hist(figsize = (15, 10))

Можно заметить, что здесь часто публикуется объявления - вторник, и примерное одинаковое количество публикации у вторника - четверг. Далее, приближенное одинаковое число по публикации - среда, понедельник и пятница, но меньше по количеству предыдующей публикации (вторник и четверг).
Число публикации в субооту меньше остальных по количеству, но не уступает количеству воскресенья (воскресенье - самое малое количество по публикации объявлений)

In [53]:
data['month_of_publication'].hist(figsize = (15, 10))

Можно заметить, что здесь часто публикуется объявления - август. Чуть меньше по количеству - март. Далее, усредненное количество по числу публикации занимает - апрель, и его приближенное значение июнь.
Самое малое количество объявление - май

In [54]:
data['days_exposition'].hist(bins = 50, figsize = (15, 10))

Можно заметить, что здесь часто продавались квартиры с момента снятия публикации - около 100 дней. Далее резко снизилась частота по продажам - около 200 дней. Данный график можно рассмотреть как распределение Пуассона. Чем дольше висит в объявлениях о продажах, тем ниже шанс продавать.

In [55]:
print('Средний срок дни по продажам дома:', data['days_exposition'].mean())
print('Медианный срок дни по продажам дома:', data['days_exposition'].median())

Средний срок дни, это говорит о том, что любой дом может снять с публикации о продажи в течение около 193 дней.
По этому сроку можно рассматривать как обычный и средний срок по продажам.

А медианный срок дни по прадажам составляет 105 дней, это говорит о том, что будет часто встречать в объявлениях такой срок, что дом будет снят с публикации о продажи в течение 105 дней.

In [56]:
import matplotlib.pyplot as plt

In [57]:
data['days_exposition'].describe()

In [58]:
data.boxplot(column='days_exposition', figsize = (15, 10))
plt.ylim(0, 600)

In [59]:
data['days_exposition'].hist(bins = 50, figsize = (15, 10), range = (0, 600), orientation="horizontal")

Если посмотреть на среднее время продажи дома - 193 дней, то самой быстрой продажи можно считать 2 дней (min = 2.0). А самой долгой - приблизительно 560 дней

Здесь находим факторы, больше всего влияющих на общую (полную) стоимость объекта. Для подготовки данных перед визуализацией мы используем сводные таблицы.

In [60]:
# Видим, что колонка last_price не удобная для отображения, попробуем заменить на исчисляемое как "млн"
def last_price_mln(row):
    try:
        return (row / 1000000).mean()
    except:
        return 'нет категории'

data['last_price_mln'] = data['last_price'].apply(last_price_mln)

In [61]:
cost_factors_of_object = data.pivot_table(index = ['first_day_exposition', 'type_floor'], values = ['floor', 'last_price','total_area', 'living_area', 'kitchen_area', 'rooms', 'day_of_publication', 'month_of_publication', 'year_of_publication'], aggfunc = {'type_floor': 'count', 'last_price': last_price_mln, 'total_area': 'mean', 'living_area': 'mean', 'kitchen_area': 'mean', 'rooms': 'count', 'day_of_publication': 'count', 'month_of_publication': 'count', 'year_of_publication': 'count'})
cost_factors_of_object

In [62]:
cost_factors_of_object.columns

In [63]:
# Подключаем библиотеку seaborn и назовем ее кратко как sb
import seaborn as sb

Построим графики, которые покажут зависимость цены от указанных ниже параметров. 

In [64]:
sb.pairplot(cost_factors_of_object, vars = ['last_price', 'total_area']).fig.set_size_inches(10,10)

Здесь можно заметить, что дом со стоимостью около 0 - 20 млн рублей (площадь около 15-150 м^2) является популярным в объявлении о продажах. 

Однако есть дом, который предлагает продавать с высокой стоимостью и большой вместимостью (огромной площадью)

In [65]:
sb.pairplot(cost_factors_of_object, vars = ['last_price', 'living_area']).fig.set_size_inches(10,10)

Здесь можно заметить обыдненый случай, что дом с жилой площадью около 0-20 м^2 является частым явлением в продажах

In [66]:
sb.pairplot(cost_factors_of_object, vars = ['last_price', 'kitchen_area']).fig.set_size_inches(10,10)

Здесь можно заметить обыдненый случай, как и предыдующий, что дом с кухонной площадью около 0-20 м^2 является частым явлением в продажах

Однако здесь виден разброс точек больше предыдущего, это говорит о том, что есть кухня с большой площадью и стоимостью

In [67]:
sb.pairplot(cost_factors_of_object, vars = ['last_price', 'rooms']).fig.set_size_inches(10,10)

Здесь можно заметить, что дом со стоимостью 0-20 млн рублей чаще всего имеет 1-4 комнат. 

Проверим иным способом

In [68]:
sb.pairplot(data, y_vars = ['last_price'], x_vars = ['rooms']).fig.set_size_inches(10,10)

Что и подтвердилось, здесь виден, что 2-4 комната чаще всего встречается в объявлении о проадажах

In [69]:
sb.pairplot(data, y_vars = ['last_price'], x_vars = ['type_floor']).fig.set_size_inches(10,10)

Здесь можно заметить, что дом с этажом "Другой" чаще всего публикуется в объявление о продажах. 

Однако есть дом категории "Последний", предлагающий продавать с огромной стоимостью

In [70]:
sb.pairplot(data, y_vars = ['last_price'], x_vars = ['day_of_publication']).fig.set_size_inches(10,10)

Здесь можно заметить, что реже всего проводит публикация в воскресенье. А частые - среда. 

Однако есть дом, публикующий в среду, предлагает продавать с огромной стоимостью

А у остальных примерное более менее одинаковые значения

In [71]:
sb.pairplot(data, y_vars = ['last_price'], x_vars = ['month_of_publication']).fig.set_size_inches(10,10)

Здесь можно заметить, что реже всего проводит публикация в июне и в апреле. Стоимость колеблется плавно, но нельзя отрицать, что есть выбросы. Если не учитывать эти выбросы, а только те точки, которые вплотно стоят друг друга, то график можно рассмотреть как синусиодальный с большим периодом

А частые - декабрь, но при этом имеет сильное стандартное отклонение, скорее всего это тот самый дом, который является самым дорогим



In [72]:
sb.pairplot(data, y_vars = ['last_price'], x_vars = ['year_of_publication']).fig.set_size_inches(10,10)

Здесь можно заметить, что реже всего продают в 2014. В 2017 есть объявление о продажах с большой стоимостью

Вычисляем среднюю цену одного квадратного метра в 10 населенных пунктах с наибольшим числом объявлений

In [73]:
locality_name_mean_price = data.pivot_table(index = 'locality_name', values = 'price_of_one_square_meter', aggfunc=['count', 'median'])
locality_name_mean_price.columns = ['count', 'median']
locality_name_mean_price = locality_name_mean_price.sort_values(by = 'count', ascending = False)
locality_name_mean_price.head(10)

Можно заметить, что Санкт-Петербург занимает 1 место по количеству объявлений о продажах - 7228 объявлений.

А в Колпино - последнее место по количеству объявлений о продажах - 97

In [74]:
city_spb = data[data['locality_name'] == 'санкт-петербург']
city_spb.head()

Посчитаем среднюю цену на 1 м^2 населенного пункта: Санкт-Петебург и Колпино

In [75]:
city_spb['cost_of_km'] = data[data['locality_name'] == 'санкт-петербург']['last_price'] / data[data['locality_name'] == 'санкт-петербург']['cityCenters_nearest_km']

In [76]:
city_spb = city_spb.sort_values(by = 'cityCenters_nearest_km', ascending = False)

In [77]:
city_spb = city_spb.sort_values(by = 'cost_of_km', ascending = False)

In [78]:
city_spb.head()

In [79]:
sb.pairplot(city_spb, y_vars = ['cost_of_km'], x_vars = ['cityCenters_nearest_km']).fig.set_size_inches(10,10)

Здесь можно заметить, что ближе к центре города, тем стоимость одной площади м^2 дороже. 

Здесь график получается плавно убывающим, подобие гиперболы

### Общий вывод

В объявлении о продажах мы в первую очередь обратимся дома со следующими параметрами, поскольку их очень много в объявлении: площадью около 100 м^2, с жилой площадью 0-70 м^2, кухонной площади 0-25 м^2, однокомнатной и двухкомнатной, высотой потолка 2,7 метра, с этажом ниже 5, ближе 10 км к центру города, к аэропорту - 20 км, к парку - 0,5 км. 

Обычно публикуются чаще всего во вторник, в августе. Обычно длится 100 дней в объявлении до тех пор, пока не найдется покупатель. В среднем длится 193 дней продажи в объявлении, но было зафиксировано быструю продажу - 2 дней, а самой медленной - 560 дней.

Хитом продажи можно называть тех дома с площадью 15-150 м^2 и со стоимостью 0-20 млн рублей, 2-4 комнат.

По графику завимость стоимость квартиры с годом публикации можно заметить, что стоимость колеблется, но с каждым годом средняя стоимость медленно растет